<a href="https://colab.research.google.com/github/Yashgaudani/demo/blob/main/Plant_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Define constants
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
EPOCHS = 10

# Load and preprocess data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting 80-20 for training-validation
)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Dataset/Apple___Apple_scab",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/Apple___Black_rot',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load InceptionV3 model pretrained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom top layers for plant disease classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

# Evaluate the model
validation_steps = validation_generator.samples // BATCH_SIZE
metrics = model.evaluate(validation_generator, steps=validation_steps)
print(f'Validation accuracy: {metrics[1]*100:.2f}%')

# Make predictions
test_image_path = ''
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=IMAGE_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=-1)
print(f'Predicted class: {predicted_class}')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dataset/Apple___Apple_scab'